# Split-charge equilibration in matrix notation, using the incidence matrix

This section shows how to derive the SQE equations in matrix notation. This approach does not depend on the assumption that split charges are introduced between all pairs of atoms. One can do so, but is is not needed to obtain a general implementation.

We first introduce a transformation $A$ from split charges to atomic charges:

$$
\vec{q} = \vec{q}_0 + A \vec{p}
$$

where $\vec{p}$ is a vector with unique split charges: when $p_{ij}$ is included, $p_{ji}$ must be absent. The matrix $A$ is the incidence matrix of the [oriented directed graph](https://en.wikipedia.org/wiki/Directed_graph#/media/File:Incidence_matrix_-_directed_graph.svg) of the split charges. For example, for a water molecule, it has the following form:

$$
A_\text{water}  = \left[
    \begin{array}{cc}
        +1 & 0 \\
        -1 & +1 \\
        0 & -1
    \end{array}
\right]
$$

The sum over all rows is zero to satisfy the total charge constraint. With this notation, the SQE model of Verstraelen et al. can be written as follows:

$$
E_\text{SQE} = (\vec{\chi} + A\vec{\Delta \chi})^\top \left(\vec{q}_0+A \vec{p}\right) + \frac{1}{2} \left(\vec{q}_0+A\vec{p}\right)^\top J \left(\vec{q}_0+A \vec{p}\right) + \frac{1}{2} \vec{p}^\top K \vec{p}
$$

in which the following matrices were introduced:

* $J$ is the **atomic** hardness matrix, i.e. the one from EEM.
* $K$ is a diagonal matrix with the bond hardness parameters.

We can then break out $\vec{q}_0$ and get:
$$
E_\text{SQE} = (\vec{\chi} + A\vec{\Delta \chi})^\top \vec{q}_0+(\vec{\chi} + A\vec{\Delta \chi})^\top A \vec{p} + \frac{1}{2} \left[\vec{q}_0^\top J \vec{q}_0+\vec{q}_0^\top JA \vec{p}+(A\vec{p})^\top J \vec{q}_0+ (A\vec{p})^\top JA \vec{p}\right] + \frac{1}{2} \vec{p}^\top K \vec{p}
$$

The stationary point can be found by simply taking the gradient of the SQE energy in matrix notation and setting this gradient to zero:
$$
\frac{\partial}{\partial \vec{p}} E_\text{SQE} = \vec{0}
$$
which evaluates to:
$$
A^\top (\vec{\chi} + A\vec{\Delta \chi}) + A^\top J \left(\vec{q}_0+A \vec{p}\right) + K \vec{p} = \vec{0}
$$

This has the form $M\vec{p}=R$ with

* $M=A^\top J A + K$
* $R=-A^\top (\vec{\chi} + A\vec{\Delta\chi} + J\vec{q}_0)$

A question is whether we can remove the $\vec{\chi}$ from the equation by writing

$A\vec{\Delta\chi'} = \vec{\chi} + A\vec{\Delta\chi}$

which gives

$\vec{\Delta\chi'} = A^\top (\vec{\chi} + A\vec{\Delta\chi})$

however, since the $A$ are compound-dependent matrices, this is likely not possible in general.

The following code cell implements this approach.

In [14]:
import numpy as np
from scipy.linalg import lstsq
from scipy.special import erf
from test_systems import *
from charge_utils import *

def construct_incidence(bonds, natom):
    incidence = np.zeros((natom, len(bonds)))
    incidence[bonds[:, 0], np.arange(len(bonds))] = 1
    incidence[bonds[:, 1], np.arange(len(bonds))] = -1
    assert (incidence.sum(axis=0) == 0).all()
    return incidence
    
def computed_hardness_matrix_toon(names, coords):
    """Compute the J matrix (from EEM).
    
    This is similar to calcJEEM, without Lagrange multiplies,
    and making use of vectorization techniques.
    """
    # Physical constant from https://en.wikipedia.org/wiki/Coulomb_constant
    ONE_4PI_EPS0 = 14.3996 # eV Å / e^2
    # Distance matrix
    dm = np.sqrt(sum([
        (coords[:, i:i+1] - coords[:, i])**2
        for i in (0, 1, 2)
    ]))
    # The radii
    my_radius = get_radius()
    radii = np.array([my_radius[name] for name in names])
    # The coulomb matrix for Gaussian charges
    # TODO: Incorrect radii
    hardness_matrix = 0.5*ONE_4PI_EPS0*erf(dm/np.sqrt(radii**2 + radii.T**2))
    # The diagonal elements are replaced by the atomic hardnesses.
    np.fill_diagonal(hardness_matrix, [eta[name] for name in names])
    return hardness_matrix

def computed_hardness_matrix(names, coords, model):
    """
    This is an updated version of Toons implementation of the hardness_matrix,
    correcting two errors in the calculation of J_EEM.
    """
    # Physical constant from https://en.wikipedia.org/wiki/Coulomb_constant
    ONE_4PI_EPS0 = 14.3996 # eV Å / e^2
    # Distance matrix
    dm = np.sqrt(sum([
        (coords[:, i:i+1] - coords[:, i])**2
        for i in (0, 1, 2)
    ]))
    N=len(names)
    hardness_matrix = np.zeros((N,N), dtype=float)
    my_eta = get_eta(model)
    beta   = get_beta(model)
    for i in range(N):
        hardness_matrix[i][i] = my_eta[names[i]]
        for j in range(i+1,N):
            betaij = beta[names[i]]*beta[names[j]]/np.sqrt(beta[names[i]]**2 + beta[names[j]]**2)
            Jij = 0.5*ONE_4PI_EPS0*erf(dm[i][j]*betaij)/dm[i][j]
            hardness_matrix[i][j] = Jij
            hardness_matrix[j][i] = Jij
    return hardness_matrix

def get_bond_parameters(bonds, pardict, names, signed):
    result = []
    for i, j in bonds:
        par = pardict.get((names[i], names[j]))
        if par is None:
            par = pardict.get((names[j], names[i]))
            if par is None:
                raise KeyError(f"No bond parameter for atom pair {i},{j}")
            if signed:
                par *= -1
        result.append(par)
    return np.array(result)

def solve_sqe_incidence(names, coords, qtotal, bonds, model, verbose=True):
    A = construct_incidence(bonds, len(coords))
    J = computed_hardness_matrix(names, coords, model)
    # Vector with atomic electronegativities
    my_chi = get_chi(model)
    chis = np.array([my_chi[name] for name in names])
    # Vectors with bond parameters
    try:
        K = get_bond_parameters(bonds, get_delta_eta(model), names, signed=False)
    except KeyError:
        sys.exit(1)
    try:
        delta_chis = get_bond_parameters(bonds, get_delta_chi(model), names, signed=True)
    except KeyError:
        sys.exit(1)
    # Construct the SQE equations.
    q0 = np.full(len(names), qtotal/len(names))
    M = np.diag(K) + np.dot(A.T, (np.dot(J, A)))
    R = -np.dot(A.T, chis + np.dot(J,q0) + np.dot(A, delta_chis))
    # Solve and convert to charges
    p = np.linalg.solve(M, R)
    q = np.dot(A, p) + q0
    # Report results
    if verbose:
        print("p [e]:", p)
        print("q [e]: ", q)
        qtot = np.sum(q)
    return q

def run_compound(molname, verbose, shells):
    mol = get_system(molname)
    print("%s" % molname)
    for bonds in mol["bonds"]:
        q = solve_sqe_incidence(mol["names"], mol["coords"], mol["qtotal"], bonds, "ACS-g")
    printDipole(q, mol["coords"], mol["atomnr"], False)

verbose = False
shells = False

for compound in [ "methanol", "water", "acetate" ]:
    run_compound(compound, verbose, shells)

methanol
p [e]: [-0.17110449  0.33691938 -0.16438559  0.16436908  0.17977516]
q [e]:  [-0.50802387  0.33691938 -0.33742533  0.16436908  0.16438559  0.17977516]
p [e]: [-0.33691938 -0.17110449 -0.16436908 -0.16438559 -0.17977516]
q [e]:  [-0.50802387  0.33691938 -0.33742533  0.16436908  0.16438559  0.17977516]
p [e]: [-0.33691938  0.16436908 -0.17110449 -0.16438559  0.17977516]
q [e]:  [-0.50802387  0.33691938 -0.33742533  0.16436908  0.16438559  0.17977516]
p [e]: [ 0.17110449 -0.33691938 -0.16436908 -0.16438559  0.17977516]
q [e]:  [-0.50802387  0.33691938 -0.33742533  0.16436908  0.16438559  0.17977516]
p [e]: [-0.17110449  0.33691938 -0.16438559  0.17977516  0.16436908]
q [e]:  [-0.50802387  0.33691938 -0.33742533  0.16436908  0.16438559  0.17977516]
Dipole = 2.07033 Debye
water
p [e]: [ 0.11909703 -0.11909703]
q [e]:  [ 0.11909703 -0.23819405  0.11909703]
p [e]: [0.11909703 0.11909703]
q [e]:  [ 0.11909703 -0.23819405  0.11909703]
p [e]: [-0.11909703 -0.11909703]
q [e]:  [ 0.119097